In [1]:
%matplotlib inline

import rasterio
from matplotlib import pyplot as plt
import numpy as np

def plot(array, label=None):
    """Plot a numpy array with a colorbar."""
    plt.imshow(array)
    plt.colorbar()
    if label is None:
        label = 'Raster {}'.format(array.shape)
    plt.title(label)
    plt.xlabel('Column #')
    plt.ylabel('Row #')

In [2]:
#Allow for interactive figure display
%matplotlib notebook
#%matplotlib inline

#Import necessary modules
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint as pp
import io
import requests
import cv2

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
# Create local webserver and auto handles authentication.
# TODO: Find way to authenticate anonymously
gauth.LocalWebserverAuth()

drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=307813950907-rvk9tbblercr3aklfvdrnng4uiddu715.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [4]:
from pygeotools.lib import iolib, warplib, geolib, timelib, malib
from urllib.request import urlopen
from urllib.request import urlretrieve

def open_file(blob_id):
    """ Read a file from Google Drive into memory. Returns an open (BytesIO) file-like object. """
    
    url = 'https://docs.google.com/uc?export=download'
    session = requests.Session()
    response = session.get(url, params = { 'id' : blob_id }, stream = True)
    token = _get_token(response)
    if token:
        params = { 'id' : blob_id, 'confirm' : token }
        response = session.get(url, params = params, stream = True)
    file_bytes = response.content
    return file_bytes

def _get_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def imageFromID(blob_id):
    resp = urlopen('https://docs.google.com/uc?export=download&id=' + blob_id)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

In [10]:
publicFolder = '0B5c3UTO8DDZwLVlDQnBoVC12TXM'
file_list = drive.ListFile({'q': "'0B5c3UTO8DDZwLVlDQnBoVC12TXM' in parents and trashed=false"}).GetList()
pp(file_list[3])

{'alternateLink': 'https://drive.google.com/file/d/0B5c3UTO8DDZwelhHMlI4NHByTWM/view?usp=drivesdk',
 'appDataContents': False,
 'capabilities': {'canCopy': True, 'canEdit': False},
 'copyRequiresWriterPermission': False,
 'copyable': True,
 'createdDate': '2017-09-15T05:11:18.039Z',
 'downloadUrl': 'https://doc-08-6o-docs.googleusercontent.com/docs/securesc/ihp3jp61pfp4sr6kb97raia0m2btq99s/s13i7l6saii6sp37d5p9glp2l23upilm/1536890400000/01742668982526945759/08302835381752040696/0B5c3UTO8DDZwelhHMlI4NHByTWM?e=download&gd=true',
 'editable': False,
 'embedLink': 'https://drive.google.com/file/d/0B5c3UTO8DDZwelhHMlI4NHByTWM/preview?usp=drivesdk',
 'etag': '"v8V6DvTBE_bfiGlEO9jLLdDEcEE/MTUwNTQ1MjI3ODAzOQ"',
 'explicitlyTrashed': False,
 'fileExtension': 'tif',
 'fileSize': '1315326',
 'headRevisionId': '0B5c3UTO8DDZwbkRSU2J0N05ZRlFhbFkyM1U4MnpmYXhRQlQwPQ',
 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/image/tiff',
 'id': '0B5c3UTO8DDZwelhHMlI4NHByTWM',
 'imageMediaMet

In [12]:
fileStreams = []
downURLs = []
for file in file_list:
    if file['title'].endswith('32m_trans.tif'):
        #pp(file['id'])
        #pp(file['webContentLink'])
        #fileStreams.append(imageFromID(file['id']))
        downURLs.append(urlretrieve(file['webContentLink'])[0])

'0B5c3UTO8DDZwcENvbkNQQTJMYlE'
'https://drive.google.com/uc?id=0B5c3UTO8DDZwcENvbkNQQTJMYlE&export=download'
'0B5c3UTO8DDZwSlh5amFKUDZ4T1U'
'https://drive.google.com/uc?id=0B5c3UTO8DDZwSlh5amFKUDZ4T1U&export=download'
'0B5c3UTO8DDZwZGZ1TkdWSmRQWDA'
'https://drive.google.com/uc?id=0B5c3UTO8DDZwZGZ1TkdWSmRQWDA&export=download'
'0B5c3UTO8DDZwSnp5Rk9XWG1pQ28'
'https://drive.google.com/uc?id=0B5c3UTO8DDZwSnp5Rk9XWG1pQ28&export=download'
'0B5c3UTO8DDZwQkVGQnVCdTdKcVk'
'https://drive.google.com/uc?id=0B5c3UTO8DDZwQkVGQnVCdTdKcVk&export=download'
'0B5c3UTO8DDZwQzJ6TFl5Rm9MTGM'
'https://drive.google.com/uc?id=0B5c3UTO8DDZwQzJ6TFl5Rm9MTGM&export=download'
'0B5c3UTO8DDZwSTg3Q1hxbDh6ZEE'
'https://drive.google.com/uc?id=0B5c3UTO8DDZwSTg3Q1hxbDh6ZEE&export=download'
'0B5c3UTO8DDZwUFZuUGtSbDlvOGs'
'https://drive.google.com/uc?id=0B5c3UTO8DDZwUFZuUGtSbDlvOGs&export=download'
'0B5c3UTO8DDZwR2NHYUM3M0U0VjQ'
'https://drive.google.com/uc?id=0B5c3UTO8DDZwR2NHYUM3M0U0VjQ&export=download'
'0B5c3UTO8DDZwNVlJV

In [8]:
dataset_list = warplib.memwarp_multi_fn(downURLs, extent='intersection', res='min', t_srs=downURLs[2])


Warping all inputs to the following:
Resolution: 32.0
Extent: [485646.72, 3093264.171, 485648.127, 3094831.672]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: cubic

1 of 29: /tmp/tmpxtnqwtb3
nl: 49 ns: 0 res: 32.000


RuntimeError: Attempt to create 0x49 dataset is illegal,sizes must be larger than zero.